In [1]:
import numpy as np
import keras
import keras.backend as k
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
from keras.optimizers import Adam
from keras.preprocessing import image
import cv2
import datetime

In [2]:
# UNCOMMENT THE FOLLOWING CODE TO TRAIN THE CNN FROM SCRATCH

# BUILDING MODEL TO CLASSIFY BETWEEN MASK AND NO MASK

model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [3]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        '../train/',   # put the folder path
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        '../test/',  #put the folder path
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')

model_saved=model.fit_generator(
        training_set,
        epochs=10,
        validation_data=test_set,

        )

Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


C:\Users\pravi\AppData\Local\Temp\ipykernel_11280\881288824.py:22: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_saved=model.fit_generator(


Epoch 1/10


83/83 [==============================] - 73s 852ms/step - loss: 0.4087 - accuracy: 0.8052 - val_loss: 0.1340 - val_accuracy: 0.9485
Epoch 2/10
83/83 [==============================] - 47s 569ms/step - loss: 0.1634 - accuracy: 0.9422 - val_loss: 0.2375 - val_accuracy: 0.8969
Epoch 3/10
83/83 [==============================] - 46s 552ms/step - loss: 0.1716 - accuracy: 0.9429 - val_loss: 0.1169 - val_accuracy: 0.9536
Epoch 4/10
83/83 [==============================] - 44s 527ms/step - loss: 0.1101 - accuracy: 0.9566 - val_loss: 0.0946 - val_accuracy: 0.9639
Epoch 5/10
83/83 [==============================] - 44s 533ms/step - loss: 0.0851 - accuracy: 0.9665 - val_loss: 0.0846 - val_accuracy: 0.9639
Epoch 6/10
83/83 [==============================] - 46s 554ms/step - loss: 0.0980 - accuracy: 0.9619 - val_loss: 0.1256 - val_accuracy: 0.9485
Epoch 7/10
83/83 [==============================] - 43s 524ms/step - loss: 0.0808 - accuracy: 0.9734 - val_loss: 0.0939 - val_accuracy: 0.96

### built-in model

In [4]:
model.save('mymodel.h5',model_saved)   #upload this model inside jupyter notbook

#To test for individual images

mymodel=load_model('mymodel.h5')

C:\Users\pravi\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### model accuracy

In [5]:
# Print training and validation accuracy
training_accuracy = model_saved.history['accuracy']
validation_accuracy = model_saved.history['val_accuracy']

print("Training Accuracy:", training_accuracy[-1])
print("Validation Accuracy:", validation_accuracy[-1])


Training Accuracy: 0.9680365324020386
Validation Accuracy: 0.969072163105011


In [12]:
test_image=image.load_img(r"..test\without_mask\10.jpg",target_size=(150,150,3))  #copy the path of test folder any one image
test_image=image.load_img(r"..test\with_mask\1-with-mask.jpg",target_size=(150,150,3))  #same as above 
test_image
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
mymodel.predict(test_image)[0][0]

1/1 [==============================] - 0s 48ms/step


0.0

In [13]:
# IMPLEMENTING LIVE DETECTION OF FACE MASK

mymodel=load_model('mymodel.h5')

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml') #upload this file in jupyter notebook


while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):   #oncce your webcame is on, to close enter 'q'
        break
    
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 21ms/step
